In [93]:
#packages
import sys
import re
import gensim
import numpy as np
import string
import pandas as pd
import os
import yahoo_finance
import yfinance as yf
import datetime
import matplotlib.pylab as plt
import json
import random
import os.path
from os import path

In [94]:
# roots
#root to article directory
#baseRoot = r'D:\wiki_cleaner\Whole_wikipedia\results100\Articles\Articles'
baseRoot = r'D:\mastersThesis\BitcoinSharesPrediction'
timePeriodForBitcoingAnalysing = "5y"

In [95]:
# colections
allArticlesPerDate = {}
datePriceDictionary = {}
startDate = None

In [96]:
# convert string to date format
def convertToDateFromat(stringDate):
    if len(stringDate.split('.')) != 3:
        print('Wrong format of date: '+stringDate)
        raise
    return datetime.datetime.strptime(stringDate.split('.')[2]+'-'+stringDate.split('.')[1]+'-'+stringDate.split('.')[0], '%Y-%m-%d').date() 

In [97]:
# create dictionary with key as datetime.date format and values as list of articles
def articlesSegregation(baseRoot):
    dateDictionary = {}
    articlesDictionary = {}
    baseRoot = baseRoot+ r'/Articles'
    for externalFolderName in os.listdir(baseRoot):
        for innerFileName in os.listdir(os.path.join(baseRoot, externalFolderName)):
            for fileName in os.listdir(os.path.join(os.path.join(baseRoot, externalFolderName), innerFileName)):            
                name = str(baseRoot+'/'+ externalFolderName+ '/' + innerFileName + '/' + fileName)
                if externalFolderName in dateDictionary:
                    dateDictionary[externalFolderName].append(name)
                elif externalFolderName not in dateDictionary:
                    dateDictionary[externalFolderName] = [name]  
    for date in dateDictionary:
        if date.split('.')[2][0] != '1' and date.split('.')[2][0] != '2':
            continue
        for article in dateDictionary[date]:
            if convertToDateFromat(date) in articlesDictionary:
                articlesDictionary[convertToDateFromat(date)].append(' '.join(open(article, 'r',errors='ignore',encoding="utf8").readlines()) + '\n')
            elif convertToDateFromat(date) not in articlesDictionary:
                articlesDictionary[convertToDateFromat(date)] = [' '.join(open(article, 'r',errors='ignore',encoding="utf8").readlines()) + '\n']
    return articlesDictionary

In [98]:
def createBitcoinSharesDataset(articlesDictionary,timePeriod):
    datePriceDictionary = {}
    # create connection object to bitcoin API
    bitcoinShares = yf.Ticker("BTC-USD")
    startDate = sorted(list(articlesDictionary.keys()))[0]
    # get historical market data for 5 year period
    historical_data = bitcoinShares.history(period=timePeriod)
    for i, row in historical_data.iterrows():
        if(i.date() >= startDate):
            datePriceDictionary[i.date()] = row['Close']
    return datePriceDictionary

In [99]:
# create dictionary with key as datetime.date format and values as list of articles
def prepareArticlesLabels(sharesDictionary):    
    labeledDictionary = {}
    sortedDict = sorted(list(sharesDictionary.keys()))
    for x in range(0, len(sortedDict)):
        if x == len(sortedDict) - 1:
            break
        if sharesDictionary[sortedDict[x]] > sharesDictionary[sortedDict[x+1]]:
            labeledDictionary[sortedDict[x+1]] = 'pos'
        else:
            labeledDictionary[sortedDict[x+1]] = 'neg'
    return labeledDictionary

In [100]:
def labelArticlesWitchMarketChange(segregatedArticles, markedDates):
    labeledArticleDictionary = {}
    for date in markedDates.keys():        
        if date in markedDates and date in segregatedArticles:
            labeledArticleDictionary[date] = [markedDates[date], segregatedArticles[date]]
        else:
            print('Missing dates: '+ str(date))
    return labeledArticleDictionary

In [101]:
def removeStopList(article):
    article = article.split() 
    mostCommonWordsFromWikipediaModel = ['the','of', 'and', 'in', 'a', 'to', 'was', 'is', 'for', 'as', 'on', 'with', 
                'by', 'he', 'that', 'at', 'from', 'his', 'it', 'an', 'were', 'which', 'are', 
                'also', 'be', 'this', 'had', 'has', 'or', 'first', 'their', 'its', 'one', 'new', 'but', 
                'who', 'her', 'not', 'after', 'they', 'she', 'have', 'two', 'been', 'other', 'when', 'into', 
                'during', 'all', 'time', 'may', 'more', 'there', 'school', 'would', 'years', 'over', 'only', 
                'where', 'between', 'most', 'later', 'up', 'some', 'about', 'such', 'national', 'made', 'then', 
                'him', 'out', 'world', 'city', 'three', 'used', 'can', 'state', 'while', 'known', 'united', 'under', 
                'university', 'became', 'part', 'many', 'than', 'film', 'second', 'these', 'no', 'being', 'year', 
                'including', 'born', 'through', 'states', 'american', 'both', 'team', 'season']
    for word in mostCommonWordsFromWikipediaModel:
        if word in article:
            article = [value for value in article if value != word]
    return article

In [102]:
def removePunctuation(article):
    return " ".join(" ".join(article).translate(str.maketrans('','',string.punctuation)).lower().replace('\n',' ').split()) 

In [103]:
def createJson(article,label):
    start = '{"text":['
    end = '], "label": "{lab}"'.format(lab = label) + '}'
    middle = ''
    for word in article:
        middle += '"' + str(word) + '"' +','
    middle = middle[:-1]
    json = start+ middle + end + '\n'
    return json

In [104]:
def preprocessTextData(labeleDictionary,finalRoot):
    resultRoot = finalRoot + r'/result'   
    if not path.exists(resultRoot + '.json'):
        file_object = open(resultRoot+'.json', 'a',errors='ignore',encoding="utf8")
    else:
        count = 0
        resultRoot = resultRoot + str(count)
        while path.exists(resultRoot+'.json'):
            count +=1
            resultRoot = resultRoot + str(count)
        file_object = open(resultRoot+'.json', 'a',errors='ignore',encoding="utf8")
    for key, value in labeleDictionary.items():
        article = labeleDictionary[key][1]
        label = labeleDictionary[key][0]
        article = removePunctuation(article)
        article = removeStopList(article)
        file_object.write(createJson(article,label))
    return resultRoot+'.json'

In [105]:
def tryJson(root):
    file_object = open(root, 'r',errors='ignore',encoding="utf8")
    lines = file_object.readlines()
    for line in lines:        
        json.loads(line)

In [112]:
def saveJson(dataSet,root,name):
    file_object = open(root +'/'+name+'.json', 'w',errors='ignore',encoding="utf8")
    file_object.write(' '.join(dataSet))
    file_object.close()

In [107]:
def splitData(ArticleList,trainProcent,testProcent,validationProcent):
    trainLastIndex = int(trainProcent*len(ArticleList))
    testFirstIndex = trainLastIndex 
    testLastIndex = testFirstIndex + int(testProcent*len(ArticleList))
    validationFirstIndex = testLastIndex 
    trainArticles = ArticleList[:trainLastIndex]
    testArticles = ArticleList[testFirstIndex:testLastIndex]
    valdiationArticles = ArticleList[validationFirstIndex:]
    return trainArticles,testArticles,valdiationArticles

In [110]:
#create 70/15/15 sets - train/validation/test
def createTrainValidationTestSets(jsonRoot,baseRoot):
    file_object = open(jsonRoot, 'r',errors='ignore',encoding="utf8")
    lines = file_object.readlines()
    random.shuffle(lines)
    posArticles = []
    negArticles = []
    for line in lines:        
        if '"pos"' in line:
            posArticles.append(line)
        elif '"neg"' in line:
            negArticles.append(line)
    trainPosArticles,testPosArticles,validationPosArticles = splitData(posArticles,0.8,0.15,0.15)
    trainNegArticles,testNegArticles,validationNegArticles = splitData(negArticles,0.8,0.15,0.15)
    saveJson(trainPosArticles + trainNegArticles,baseRoot,'train')
    saveJson(testPosArticles + testNegArticles,baseRoot,'test')
    saveJson(validationPosArticles + validationNegArticles,baseRoot,'validation')
    #return (trainPosArticles + trainNegArticles),(testPosArticles + testNegArticles),(validationPosArticles + validationNegArticles)

In [113]:
segregatedArticles = articlesSegregation(baseRoot)
bitcoinSharesPrice = createBitcoinSharesDataset(segregatedArticles,timePeriodForBitcoingAnalysing)
markedDates = prepareArticlesLabels(bitcoinSharesPrice)
labeledArticlesDictionary = labelArticlesWitchMarketChange(segregatedArticles, markedDates)
jsonRoot = preprocessTextData(labeledArticlesDictionary,baseRoot)
createTrainValidationTestSets(jsonRoot,baseRoot)
#train,test,validation = createTrainValidationTestSets(jsonRoot)

Missing dates: 2020-04-21
Missing dates: 2020-04-22
Missing dates: 2020-04-23
Missing dates: 2020-04-25
